In [1]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
# !pip install ipython-sql

In [2]:
#When the command above completes, the ibm_db library is loaded in your notebook.

import ibm_db

In [3]:
#Replace the placeholder values with the actuals for your Db2 Service Credentials


dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_hostname = "19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"  # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_port = "30699"                    # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = "prj63437"                 # e.g. "abc12345"
dsn_pwd = "HHrZHS29hKohRSpb"                 # e.g. "7dBZ3wWt9XN6$o0J"
dsn_security = "SSL"              #i.e. "SSL"

In [4]:
#Create database connection

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  BLUDB as user:  prj63437 on host:  19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud


In [5]:
#Lets first drop the table INSTRUCTOR in case it exists from a previous attempt
dropQuery = "drop table INSTRUCTOR"

#Now execute the drop statment
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

Above exception/error indicating that INSTRUCTOR is an undefined name, that's okay. It just implies that the INSTRUCTOR table does not exist in the table - which would be the case if I had not created it previously.

Exception: [IBM][CLI Driver][DB2/LINUXX8664] SQL0204N "ABC12345.INSTRUCTOR" is an undefined name. SQLSTATE=42704 SQLCODE=-204

In [6]:
#Construct the Create Table DDL statement 

createQuery="create table INSTRUCTOR(ID INTEGER PRIMARY KEY NOT NULL, FNAME VARCHAR(20), LNAME VARCHAR(20), CITY VARCHAR(20), CCODE CHAR(2))"

createStmt= ibm_db.exec_immediate(conn, createQuery)

In [7]:
#Insert data into the table

insertQuery = "insert into INSTRUCTOR values (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"

insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [8]:
insertQuery2 = "insert into INSTRUCTOR values (2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

Query data in the table

In this step we will retrieve data we inserted into the INSTRUCTOR table.

In [9]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only)

ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA',
 4: 'CA'}

In [10]:
#Fetch the rest of the rows and print the ID and FNAME for those rows

while ibm_db.fetch_both(selectStmt) != False:
    print("ID : ",ibm_db.result(selectStmt, 0), "FNAME : ", ibm_db.result(selectStmt, "FNAME"))

ID :  2 FNAME :  Raul
ID :  3 FNAME :  Hima


In [11]:
#now write and execute an update statement that changes the Rav's CITY to MOOSETOWN

updateQuery="update INSTRUCTOR set CITY='MOOSETOWN' where FNAME='Rav'"

updateStmt=ibm_db.exec_immediate(conn, updateQuery)

In [12]:
#Construct the query that retrieves all rows from the INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only)
ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'MOOSETOWN',
 3: 'MOOSETOWN',
 'CCODE': 'CA',
 4: 'CA'}

Retrieve data into Pandas

In [13]:
import pandas as pd
import ibm_db_dbi

In [14]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [15]:
#query statement to retrieve all rows in INSTRUCTOR table

selectQuery="select * from INSTRUCTOR"

#retrieve the query results into a pandas dataframe

pdf= pd.read_sql(selectQuery, pconn)

#print just the LNAME for first row in the pandas data frame

pdf.LNAME[0]

'Ahuja'

In [16]:
pdf

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [17]:
#shape method to see how many rows and columns are in the dataframe

pdf.shape

(3, 5)

In [18]:
#Close the Connection

ibm_db.close(conn)

True